In [11]:
import numpy as np
import sys
import os
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
sys.path.append(os.path.abspath('C:/Users/vpming/tuni_ml/src'))
from extract_data import build_cellwise_df
import torch.nn as nn

In [12]:
data_path = 'C:/Users/vpming/tuni_ml/data'
df = build_cellwise_df(data_path)

In [13]:
x = np.stack(df['time_trace'])
print(x.shape)
y = df['dis_to_target'].values
print(y.reshape(-1,1))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_scaler = StandardScaler()
x_train_s = x_scaler.fit_transform(x_train)
x_test_s = x_scaler.transform(x_test)

y_scaler = StandardScaler()
y_train_s = y_scaler.fit_transform(y_train.reshape(-1,1)).ravel()
y_test_s = y_scaler.transform(y_test.reshape(-1,1)).ravel()

(1750, 1001)
[[3]
 [3]
 [2]
 ...
 [2]
 [2]
 [3]]


In [14]:
train_ds = TensorDataset(torch.tensor(x_train_s, dtype=torch.float32),
                         torch.tensor(y_train_s, dtype=torch.float32))

test_ds = TensorDataset(torch.tensor(x_test_s, dtype=torch.float32),
                        torch.tensor(y_test_s, dtype=torch.float32))

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

In [15]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
    def forward(self, x):
        return self.net(x).squeeze(-1)

In [22]:
model = MLP(x_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1:2d}: Train loss {running_loss/len(train_loader):.4f}")

torch.save({
    'model_state': model.state_dict(),
    'scaler_x': x_scaler,
    'scaler_y': y_scaler
}, 'C:/Users/vpming/tuni_ml/src/model/mlp_dtt.pt')


Epoch  1: Train loss 0.7237
Epoch  2: Train loss 0.4744
Epoch  3: Train loss 0.3796
Epoch  4: Train loss 0.2983
Epoch  5: Train loss 0.2539
Epoch  6: Train loss 0.2150
Epoch  7: Train loss 0.1799
Epoch  8: Train loss 0.1564
Epoch  9: Train loss 0.1375
Epoch 10: Train loss 0.1263
Epoch 11: Train loss 0.1040
Epoch 12: Train loss 0.1001
Epoch 13: Train loss 0.0969
Epoch 14: Train loss 0.0812
Epoch 15: Train loss 0.0782
Epoch 16: Train loss 0.0645
Epoch 17: Train loss 0.0587
Epoch 18: Train loss 0.0638
Epoch 19: Train loss 0.0550
Epoch 20: Train loss 0.0473


In [23]:
checkpoint = torch.load('C:/Users/vpming/tuni_ml/src/model/mlp_dtt.pt', weights_only=False)
model = MLP(x_train.shape[1])
model.load_state_dict(checkpoint['model_state'])
model.eval()

scaler_x = checkpoint['scaler_x']
scaler_y = checkpoint['scaler_y']

In [28]:
import h5py
file_path = 'C:/Users/vpming/tuni_ml\data\stim_0.5_beta_0.04_noise_0.04_kcross_0.0050\sim_data__stimMag_0.50_beta_0.40_noise_0.040_kcross_0.0050_nSamples_1000_5.h5'

with h5py.File(file_path,'r') as f:
    x_new = np.array(f['timeTraces']['2']).T

x_new_scale = scaler_x.transform(x_new)
with torch.no_grad():
    pred_scaled = model(torch.tensor(x_new_scale, dtype=torch.float32)).numpy()
pred = scaler_y.inverse_transform(pred_scaled.reshape(-1,1)).ravel()
print(pred)

[2.2558973 2.1968734 2.3069746 1.7485405 1.9559821 3.1011918 1.9392272
 1.5799947 2.6926613 2.287417  2.5216088 1.5297105 0.9830334 1.3135597
 1.35044   2.6895366 2.0199523 1.4634914 1.5492975 1.6825321 2.0958734
 1.6111461 1.3420255 2.3264484 2.3807058]
